# Setup A3D domain
This notebook provides a workflow to set up an Alpine3D simulation for Antarctica.
- It reads the file ```a3d_settings.txt``` for the settings
- Creates the Digital Elevation Model (DEM)
- Creates a list of stand-alone SNOWPACK simulations to initialize the grid points
- Creates a grid mapping the SNOWPACK simulations to the grid points (LUS-file)

In [ ]:
import os
import re
import numpy as np
from scipy.interpolate import NearestNDInterpolator
import rasterio
import pyproj

from shapely.geometry import Polygon, Point, shape
from shapely.ops import transform
import shapely.prepared

# Initialize required projections
wgs84 = pyproj.CRS('EPSG:4326')  # WGS 84
aps = pyproj.CRS('EPSG:3031')    # Antarctic Polar Stereographic
gps = pyproj.CRS('EPSG:3413')    # Greenland Polar Stereographic

project_to_aps = pyproj.Transformer.from_crs(wgs84, aps, always_xy=True).transform
project_to_gps = pyproj.Transformer.from_crs(wgs84, gps, always_xy=True).transform

# Import settings

In [ ]:
exec(open("a3d_settings.txt").read())

## Read SNOWPACK points
SNOWPACK.pts should contain `latitude longitude` for all SNOWPACK run points

In [ ]:
%%bash
#cat <(tail -n+2 MERRA-2_AIS_subset_icefr80.txt | tr ',' ' ') <(grep ^VSTATION greenland_mask50percent.lst | awk '{print $4, $5}') > SNOWPACK.pts

In [ ]:
# Read points for which we performed SNOWPACK simulations
SP_pts = np.loadtxt(SNOWPACK_pts, comments="#", delimiter=" ", unpack=False)

# Now transform coordinates
SP_pts_A = []    # WGS 84
SP_pts_G = []    # WGS 84
SP_pts_AT = []   # Antarctic Polar Stereographic
SP_pts_GT = []   # Greenland Polar Stereographic
for i in SP_pts:
    pt = Point(i[1], i[0])                  # WGS-84, EPSG:4326
    if i[0] < 0:                            # Southern hemisphere
        SP_pts_t = transform(project_to_aps, Point(i[1], i[0]))
        SP_pts_A.append([i[1], i[0]])
        SP_pts_AT.append([SP_pts_t.x, SP_pts_t.y])
    else:
        SP_pts_t = transform(project_to_gps, Point(i[1], i[0]))
        SP_pts_G.append([i[1], i[0]])
        SP_pts_GT.append([SP_pts_t.x, SP_pts_t.y])
SP_pts_A = np.array(SP_pts_A)
SP_pts_AT = np.array(SP_pts_AT)
SP_pts_G = np.array(SP_pts_G)
SP_pts_GT = np.array(SP_pts_GT)

## Function to write ASCII grids

In [ ]:
def writeGrid(grid, filename):
    nrows = np.shape(Z)[0]
    ncols = np.shape(Z)[1]
    nodata = -9999
    xll = dem_ulx
    yll = dem_lry
    f = open(filename, 'w')
    f.write("ncols " + str(ncols) + "\n")
    f.write("nrows " + str(nrows) + "\n")
    f.write("xllcorner " + str(xll) + "\n")
    f.write("yllcorner " + str(yll) + "\n")
    f.write("cellsize " + str(cellsize) + "\n")
    f.write("NODATA_value " + str(nodata) + "\n")
    np.savetxt(f, grid, fmt="%.0f")
    f.close()

# Determine the area represented by a SNOWPACK grid point based on nearest neighbor interpolation
Here, we first create the Digital Elevation Model (DEM), after which we apply nearest neighbor interpolation using the SNOWPACK grid points index to identify which SNOWPACK grid point is closest to each of the grid cells in the DEM. We generate a file called a Land-Use-File (LUS), which determines which *.sno file Alpine3D will read in for each grid point.

In [ ]:
#
# Antarctica
#

# First, create DEM
tgt_dem = input_grids_prefix + ".dem"
toexec = "gdal_translate -of AAIGrid -a_nodata -9999 -tr " + str(cellsize) + " " + str(cellsize) + " -projwin " + str(ulx) + " " + str(uly) + " " + str(lrx) + " " + str(lry) + " " + str(src_dem) + " " + str(tgt_dem)
os.system(toexec)

# Now read the bounds of the created DEM, to make sure the LUS file corresponds exactly to the DEM
dem = rasterio.open(input_grids_prefix + ".dem")
dem_ulx = dem.bounds[0];
dem_lry = dem.bounds[1];
dem_lrx = dem.bounds[2];
dem_uly = dem.bounds[3];
dem.close();

# Then, determine mapping of SNOWPACK standalone simulations to the requested A3D domain
x = SP_pts_AT[:,0]                       # Select all X-coordinates from SNOWPACK-run points
y = SP_pts_AT[:,1]                       # Select all Y-coordinates from SNOWPACK-run points
z = [i for i in range(len(SP_pts_AT))]   # The z-value is set as the index of the grid point

XX = np.arange(dem_ulx, dem_lrx, cellsize)
YY = np.arange(dem_lry, dem_uly, cellsize)

XX = np.arange(dem_ulx, dem_lrx, cellsize)
YY = np.arange(dem_lry, dem_uly, cellsize)

# 2D grid for interpolation, note that we shift from cell corners to cell centers for interpolation!
X, Y = np.meshgrid(XX + cellsize/2, YY+cellsize/2)

#
# We now interpolate the SNOWPACK indices with nearest neighbor
# This will be the "LUS", or land-use-file, mapping which *sno file Alpine3D should use for each pixel
#
interp = NearestNDInterpolator(list(zip(x, y)), z)                 # Construct interpolator
Z = interp(X, Y)

# Write the grid
writeGrid(Z, input_grids_prefix + ".lus")

# Write the list of index, lat, lon
f = open(input_grids_prefix + ".lst", 'w')
f.write("#index latitude longitude easting northing\n")
for idx in np.unique(Z):
    f.write('{:d}'.format(idx) + " " + '{:.3f}'.format(SP_pts_A[idx,1]) + " " + '{:.3f}'.format(SP_pts_A[idx,0]) + " " + '{:.0f}'.format(SP_pts_AT[idx,0]) + " " + '{:.0f}'.format(SP_pts_AT[idx,1]) + "\n")
f.close()

In [ ]:
# Plot interpolated field
import matplotlib.pyplot as plt
plt.pcolormesh(X, Y, Z, shading='auto')
#plt.plot(x, y, "ok", label="input point")
plt.legend()
plt.colorbar()
plt.axis("equal")
plt.show()

# Create list of meteo points
We now create a list of meteo stations to include in the simulations, based on the SNOWPACK stand-alone grid points, assuming that SNOWPACK stand-alone was run for each grid point for which meteo data is available. Note that in our study, we use a subsetted MERRA-2 grid based on climatological similarities between neighboring grid points.

In [ ]:
# Apply meteo margin
meteo_ulx = dem_ulx - meteo_margin
meteo_uly = dem_uly + meteo_margin
meteo_lrx = dem_lrx + meteo_margin
meteo_lry = dem_lry - meteo_margin

# Now loop over SNOWPACK points to determine if we are going to use the meteo forcing
x = SP_pts_AT[:,0]                       # Select all X-coordinates from SNOWPACK-run points
y = SP_pts_AT[:,1]                       # Select all Y-coordinates from SNOWPACK-run points
z = [i for i in range(len(SP_pts_AT))]   # The z-value is set as the index of the grid point

XX = np.arange(meteo_ulx, meteo_lrx, cellsize)
YY = np.arange(meteo_lry, meteo_uly, cellsize)

XX = np.arange(meteo_ulx, meteo_lrx, cellsize)
YY = np.arange(meteo_lry, meteo_uly, cellsize)

# 2D grid for interpolation, note that we shift from cell corners to cell centers for interpolation!
X, Y = np.meshgrid(XX + cellsize/2, YY+cellsize/2)

#
# We now interpolate the SNOWPACK indices with nearest neighbor to the Alpine3D grid including the
# meteo_margin
#
interp = NearestNDInterpolator(list(zip(x, y)), z)                 # Construct interpolator
Z = interp(X, Y)

# Write the list of index, lat, lon
f = open(meteofiles_prefix + "meteostn.lst", 'w')
f.write("#index latitude longitude easting northing\n")
for idx in np.unique(Z):
    f.write('{:d}'.format(idx) + " " + '{:.3f}'.format(SP_pts_A[idx,1]) + " " + '{:.3f}'.format(SP_pts_A[idx,0]) + " " + '{:.0f}'.format(SP_pts_AT[idx,0]) + " " + '{:.0f}'.format(SP_pts_AT[idx,1]) + "\n")
f.close()